# Exploration of SQLAlchemy 

In [1]:
from sqlalchemy import create_engine,  MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

Base = declarative_base()
engine = create_engine('sqlite:///testSQLAlchemy.db', echo=False, 
						connect_args={'check_same_thread':False})
metadata_obj = MetaData()
Session = sessionmaker(bind=engine)

session = Session()

In [2]:
# def query_result(query_obj):
    

### CREATE TABLES <br> usually in the "models.py"

In [4]:
from sqlalchemy import Column, String, Integer, ForeignKey
from sqlalchemy.orm import relationship

class SubjectGroup(Base):
    __tablename__ = "subject_group"
    
    group_id        = Column(Integer, autoincrement=True, primary_key=True)
    group_descript  = Column(String)
    
    repository      = relationship("Repository")
#     def __repr__(self):
#             return f"{group_id}"
   
    
class Repository(Base):
    __tablename__ = "repository"
    
    id         = Column(Integer, autoincrement=True, primary_key=True)
    group_id   = Column(Integer, ForeignKey("subject_group.group_id"))
    repo_name  = Column(String)
    repo_link  = Column(String)
#     def __repr__(self):
#             return f"{repo_name}"


	
# always initialise with create_all
Base.metadata.create_all(engine)


### INITIALISE DUMMY DATA

#### Field Vectors

In [5]:
descript_vect = ['Male London', 
                 'Female Surrey', 
                 'Health Workers', 
                 'Elder subjects', 
                 'Liverpool group', 
                 'Subjects over 80', 
                 'Male Surrey', 
                 'Health Workers 40']

In [6]:
repo_name_vect = [
    ['PRIDE','MetaboLight'], 
    ['ZENODO'], 
    ['PRIDE','ZENODO'], 
    ['PRIDE','MetaboLight','ZENODO'],
    ['PRIDE', 'ZENODO'], 
    ['MetaboLight', 'ZENODO'], 
    ['PRIDE', 'MetaboLight', 'ZENODO'], 
    ['PRIDE', 'MetaboLight', 'ZENODO']]

In [7]:
repo_link_vect = [
    ['link1','link2'], 
    ['link1'], 
    ['link1','link2'], 
    ['link1','link2','link3'],
    ['link1','link2'], 
    ['link1','link2'], 
    ['link1','link2','link3'], 
    ['link1','link2','link3']]

In [8]:
subject_group_table   = SubjectGroup
repository_table = Repository

for ind_group, description in enumerate(descript_vect):
    new_group = subject_group_table(group_descript = description)
    session.add(new_group)
    session.commit()
    print(new_group.group_id)
    
    for ind_link, repository in enumerate(repo_name_vect[ind_group]):
        new_repo = repository_table(group_id = new_group.group_id,
                                    repo_name  = repository,
                                    repo_link  = repo_link_vect[ind_group][ind_link])
        session.add(new_repo)
        session.commit()



1
2
3
4
5
6
7
8


### SHOW TABLES

In [9]:
import pandas as pd


group_table = session.query(SubjectGroup)
repo_table = session.query(Repository)


df = pd.DataFrame(columns=['group_id', 'description'])     
for group in group_table.all():
    new_row = {'group_id'  : str(group.group_id),
               'description' : group.group_descript 
               }
    
    df=df.append(new_row, ignore_index=True)
df.set_index('group_id', inplace=True, drop=True)
display(df)

df = pd.DataFrame(columns=['id', 'group_id', 'repo_name', 'repo_link'])
for repo in repo_table.all():
#     print('\t'.join([str(repo.id),
#                      str(repo.group_id),
#                          repo.repo_name, 
#                          repo.repo_link])
    new_row = {'id'        : str(repo.id),
               'group_id'  : str(repo.group_id),
               'repo_name' : repo.repo_name, 
               'repo_link' : repo.repo_link}
    
    df=df.append(new_row, ignore_index=True)
df.set_index('id', inplace=True, drop=True)
display(df)


df = pd.DataFrame(columns=['group_id', 'group_description', 'repo_id', 'repo_name', 'repo_link'])
for group in group_table.all():
#     print('\t'.join([str(group.group_id), 
#                      group.group_descript])
#          )
    repo_records = session.query(Repository).filter_by(group_id=group.group_id) #group_table.group_id
    for repository in repo_records.all():
        new_row = {'group_id'          : group.group_id, 
                   'group_description' : group.group_descript,
                   'repo_id'   : repository.id,
                   'repo_name' : repository.repo_name, 
                   'repo_link' : repository.repo_link
                  }
        df=df.append(new_row, ignore_index=True)
df.set_index('group_id', inplace=True, drop=True)
display(df)

# repo_records = session.query(Repository).filter_by(group_id=group_table[0].group_id) #group_table.group_id

# Inspect the Query object
print(repo_records.column_descriptions)
print('\n--- QUERY OBJECT - RAW COLUMNS ---')
for raw_column in repo_records._raw_columns: # Because the query is only on the table repository, there is only one column column 
    print(raw_column)
print('\n\n')


group_obj = df.groupby(['group_id', 'group_description', 'repo_name'])['repo_link'].apply(list)
df= pd.DataFrame(group_obj)
#df.set_index('group_id', inplace=True, drop=True)
display(df)

,description
group_id,
1,Male London
2,Female Surrey
3,Health Workers
4,Elder subjects
5,Liverpool group
6,Subjects over 80
7,Male Surrey
8,Health Workers 40


,group_id,repo_name,repo_link
id,,,
1,1,PRIDE,link1
2,1,MetaboLight,link2
3,2,ZENODO,link1
4,3,PRIDE,link1
5,3,ZENODO,link2
6,4,PRIDE,link1
7,4,MetaboLight,link2
8,4,ZENODO,link3
9,5,PRIDE,link1


,group_description,repo_id,repo_name,repo_link
group_id,,,,
1,Male London,1,PRIDE,link1
1,Male London,2,MetaboLight,link2
2,Female Surrey,3,ZENODO,link1
3,Health Workers,4,PRIDE,link1
3,Health Workers,5,ZENODO,link2
4,Elder subjects,6,PRIDE,link1
4,Elder subjects,7,MetaboLight,link2
4,Elder subjects,8,ZENODO,link3
5,Liverpool group,9,PRIDE,link1


[{'name': 'Repository', 'type': <class '__main__.Repository'>, 'aliased': False, 'expr': <class '__main__.Repository'>, 'entity': <class '__main__.Repository'>}]

--- QUERY OBJECT - RAW COLUMNS ---
repository





repo_link
group_id group_description repo_name            
1        Male London       MetaboLight   [link2]
                           PRIDE         [link1]
2        Female Surrey     ZENODO        [link1]
3        Health Workers    PRIDE         [link1]
                           ZENODO        [link2]
4        Elder subjects    MetaboLight   [link2]
                           PRIDE         [link1]
                           ZENODO        [link3]
5        Liverpool group   PRIDE         [link1]
                           ZENODO        [link2]
6        Subjects over 80  MetaboLight   [link1]
                           ZENODO        [link2]
7        Male Surrey       MetaboLight   [link2]
                           PRIDE         [link1]
                           ZENODO        [link3]
8        Health Workers 40 MetaboLight   [link2]
                           PRIDE         [link1]
                           ZENODO        [link3]

### ADD RECORD

In [10]:
# ADD new GROUP with REPOSITORIES
new_group = { 
                'group_descript' :  'Young Male Wessex',
                'repo_name'      : ['PRIDE', 'ZENODO'],
                'repo_link'      : ['link_PRIDE', 'link_ZENODO']
            }

subject_group_table   = SubjectGroup
repository_table = Repository

# Submit new SUBJECT GROUP
new_group_to_submit = subject_group_table(group_descript = new_group['group_descript'])
session.add(new_group_to_submit)
session.commit()

# Submit new REPOSITORIES
for ind_link, repo in enumerate(new_group['repo_name']):
    new_repo_to_submit = repository_table(group_id   = new_group_to_submit.group_id,
                                          repo_name  = repo,
                                          repo_link  = new_group['repo_link'][ind_link]
                                         )
                                            
    session.add(new_repo_to_submit)
    session.commit()

### DELETE RECORD

### QUERY

In [11]:
# How do you query a one-to-many relationship in an SQLAlchemy object instance?
# https://stackoverflow.com/questions/32016827/how-do-you-query-a-one-to-many-relationship-in-an-sqlalchemy-object-instance
query_result = session.query(SubjectGroup, Repository).filter(SubjectGroup.group_id == Repository.group_id).filter(Repository.repo_name == 'ZENODO')

In [12]:
print(query_result)
list_dir=list(dir(query_result))
print('\n--- QUERY OBJECT - DIR ---')
for dir_feature in list_dir:
    if 'col' in dir_feature: print(dir_feature)
print('\n--- QUERY OBJECT - RAW COLUMNS ---')
for raw_column in query_result._raw_columns:
    print(raw_column)
# print(query_result[0])
# for record in query_result.all():
#      pass

SELECT subject_group.group_id AS subject_group_group_id, subject_group.group_descript AS subject_group_group_descript, repository.id AS repository_id, repository.group_id AS repository_group_id, repository.repo_name AS repository_repo_name, repository.repo_link AS repository_repo_link 
FROM subject_group, repository 
WHERE subject_group.group_id = repository.group_id AND repository.repo_name = ?

--- QUERY OBJECT - DIR ---
_raw_columns
add_column
add_columns
column_descriptions

--- QUERY OBJECT - RAW COLUMNS ---
subject_group
repository


In [14]:

for record in query_result.all():
    print(record[0].group_id, record[0].group_descript, 
          record[1].id, record[1].group_id, record[1].repo_name, record[1].repo_link)
    new_row

2 Female Surrey 3 2 ZENODO link1
3 Health Workers 5 3 ZENODO link2
4 Elder subjects 8 4 ZENODO link3
5 Liverpool group 10 5 ZENODO link2
6 Subjects over 80 12 6 ZENODO link2
7 Male Surrey 15 7 ZENODO link3
8 Health Workers 40 18 8 ZENODO link3
9 Young Male Wessex 20 9 ZENODO link_ZENODO
